## Imports and database connection

In [1]:
import pandas as pd
import psycopg2
import csv

from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

# File 'database_configuration.txt' should contain database configurations in csv format like:
# dialect,postgresql+psycopg2'
# user,postgres
# password,password
# host,localhost
# port,5432
# database,database_name

with open('database_configuration.txt', mode='r') as infile:
    reader = csv.reader(infile)
    database_configuration = {rows[0]:rows[1] for rows in reader}
    
dialect = database_configuration["dialect"]
database = database_configuration["database"]
user = database_configuration["user"]
password = database_configuration["password"]
host = database_configuration["host"]
port = database_configuration["port"]

## Create schema and tables

In [2]:
conn = psycopg2.connect(database=database, user=user, password=password, host=host, port=port)

conn.autocommit = True
cursor = conn.cursor()

sql = '''
        -- SCHEMA: public

        DROP SCHEMA IF EXISTS public CASCADE ;

        CREATE SCHEMA IF NOT EXISTS public
            AUTHORIZATION postgres;

        COMMENT ON SCHEMA public
            IS 'standard public schema';

        GRANT ALL ON SCHEMA public TO PUBLIC;

        GRANT ALL ON SCHEMA public TO postgres;
        
        -- Table: public.comune

        DROP TABLE IF EXISTS public.comune;

        CREATE TABLE IF NOT EXISTS public.comune
        (
            id integer NOT NULL,
            nome character varying(255) COLLATE pg_catalog."default" NOT NULL,
            cap character varying(255) COLLATE pg_catalog."default",
            provincia character varying(255) COLLATE pg_catalog."default",
            regione character varying(255) COLLATE pg_catalog."default",
            CONSTRAINT comune_pk PRIMARY KEY (id)
        )

        TABLESPACE pg_default;

        ALTER TABLE IF EXISTS public.comune
            OWNER to postgres;
            
        -- Table: public.azienda

        DROP TABLE IF EXISTS public.azienda;

        CREATE TABLE IF NOT EXISTS public.azienda
        (
            id integer NOT NULL,
            n_registro integer NOT NULL,
            nome character varying(255) COLLATE pg_catalog."default" NOT NULL,
            data_deposito date NOT NULL,
            luogo_deposito character varying(255) COLLATE pg_catalog."default",
            settore_produzione character varying(255) COLLATE pg_catalog."default" NOT NULL,
            importazione character varying(255) COLLATE pg_catalog."default",
            nome_marchio character varying(255) COLLATE pg_catalog."default",
            data_registrazione_marchio date NOT NULL,
            descrizione_marchio text COLLATE pg_catalog."default",
            CONSTRAINT azienda_pk PRIMARY KEY (id)
        )

        TABLESPACE pg_default;

        ALTER TABLE IF EXISTS public.azienda
            OWNER to postgres;
            
        -- Table: public.luogo_di_interesse

        DROP TABLE IF EXISTS public.luogo_di_interesse;

        CREATE TABLE IF NOT EXISTS public.luogo_di_interesse
        (
            id integer NOT NULL,
            nome character varying(255) COLLATE pg_catalog."default",
            tipologia character varying(255) COLLATE pg_catalog."default",
            secolo_costruzione character varying(255) COLLATE pg_catalog."default",
            prima_attestazione character varying(255) COLLATE pg_catalog."default",
            stile character varying(255) COLLATE pg_catalog."default",
            CONSTRAINT luogo_di_interesse_pk PRIMARY KEY (id)
        )

        TABLESPACE pg_default;

        ALTER TABLE IF EXISTS public.luogo_di_interesse
            OWNER to postgres;
        
        -- Table: public.itinerario_aziendale

        DROP TABLE IF EXISTS public.itinerario_aziendale;

        CREATE TABLE IF NOT EXISTS public.itinerario_aziendale
        (
            id integer NOT NULL,
            tipologia_itinerario character varying(255) COLLATE pg_catalog."default" NOT NULL,
            comune_id integer NOT NULL,
            localita character varying(255) COLLATE pg_catalog."default",
            latitudine numeric(8,6),
            longitudine numeric(9,6),
            note text COLLATE pg_catalog."default",
            bibliografia character varying(255) COLLATE pg_catalog."default",
            sitografia character varying(255) COLLATE pg_catalog."default",
            url_immagine character varying(255) COLLATE pg_catalog."default",
            azienda_id integer NOT NULL,
            CONSTRAINT itinerario_aziendale_pk PRIMARY KEY (id),
            CONSTRAINT itinerario_aziendale_azienda FOREIGN KEY (azienda_id)
                REFERENCES public.azienda (id) MATCH SIMPLE
                ON UPDATE NO ACTION
                ON DELETE NO ACTION,
            CONSTRAINT itinerario_aziendale_comune FOREIGN KEY (comune_id)
                REFERENCES public.comune (id) MATCH SIMPLE
                ON UPDATE NO ACTION
                ON DELETE NO ACTION
        )

        TABLESPACE pg_default;

        ALTER TABLE IF EXISTS public.itinerario_aziendale
            OWNER to postgres;
            
        -- Table: public.itinerario_culinario

        DROP TABLE IF EXISTS public.itinerario_culinario;

        CREATE TABLE IF NOT EXISTS public.itinerario_culinario
        (
            id integer NOT NULL,
            nome character varying(255) COLLATE pg_catalog."default" NOT NULL,
            comune_id integer NOT NULL,
            tipologia character varying(255) COLLATE pg_catalog."default",
            descrizione text COLLATE pg_catalog."default",
            note text COLLATE pg_catalog."default",
            url_immagine character varying(255) COLLATE pg_catalog."default",
            latitudine numeric(8,6),
            longitudine numeric(9,6),
            CONSTRAINT itinerario_culinario_pk PRIMARY KEY (id),
            CONSTRAINT itinerario_culinario_comune FOREIGN KEY (comune_id)
                REFERENCES public.comune (id) MATCH SIMPLE
                ON UPDATE NO ACTION
                ON DELETE NO ACTION
        )

        TABLESPACE pg_default;

        ALTER TABLE IF EXISTS public.itinerario_culinario
            OWNER to postgres;

        -- Table: public.itinerario_culturale

        DROP TABLE IF EXISTS public.itinerario_culturale;

        CREATE TABLE IF NOT EXISTS public.itinerario_culturale
        (
            id integer NOT NULL,
            luogo_interesse_id integer NOT NULL,
            comune_id integer NOT NULL,
            via character varying(255) COLLATE pg_catalog."default",
            localita character varying(255) COLLATE pg_catalog."default",
            tipologia character varying(255) COLLATE pg_catalog."default",
            informazioni text COLLATE pg_catalog."default",
            note text COLLATE pg_catalog."default",
            bibliografia character varying(255) COLLATE pg_catalog."default",
            sitografia character varying(255) COLLATE pg_catalog."default",
            url_immagine character varying(255) COLLATE pg_catalog."default",
            latitudine numeric(8,6),
            longitudine numeric(9,6),
            CONSTRAINT itinerario_culturale_pk PRIMARY KEY (id),
            CONSTRAINT itinerario_comune FOREIGN KEY (comune_id)
                REFERENCES public.comune (id) MATCH SIMPLE
                ON UPDATE NO ACTION
                ON DELETE NO ACTION,
            CONSTRAINT itinerario_culturale_luogo_di_interesse FOREIGN KEY (luogo_interesse_id)
                REFERENCES public.luogo_di_interesse (id) MATCH SIMPLE
                ON UPDATE NO ACTION
                ON DELETE NO ACTION
        )

        TABLESPACE pg_default;

        ALTER TABLE IF EXISTS public.itinerario_culturale
            OWNER to postgres;
        
        '''
cursor.execute(sql)
conn.commit()
conn.close()


## Populate the database

### Import tables

In [3]:
itineraries_caserta_df = pd.read_pickle("../Dataset_clean/itineraries-Caserta.pkl")
itineraries_costiera_amalfitana_df = pd.read_pickle("../Dataset_clean/itineraries-CostieraAmalfitana.pkl")
itineraries_nocera_df = pd.read_pickle("../Dataset_clean/itineraries-Nocera.pkl")
itineraries_nocera_inferiore_df = pd.read_pickle("../Dataset_clean/itineraries-NoceraInferiore.pkl")

itineraries_costiera_amalfitana_df = itineraries_costiera_amalfitana_df.rename(str.capitalize, axis='columns')

In [4]:
itineraries_caserta_df.columns

Index(['Categoria itinerario', 'Nome', 'Comune', 'Località', 'Via', 'Cap',
       'Provincia', 'Regione', 'Tipologia', 'Secolo di costruzione',
       'Anno prima attestazione', 'Itinerario (periodo)', 'Stile',
       'Informazioni', 'URL Immagine', 'Bibliografia', 'Sitografia',
       'Rifunzionalizzato', 'Tipo di Rifunzionalizzazione', 'Latitude',
       'Longitude'],
      dtype='object')

In [5]:
itineraries_costiera_amalfitana_df.columns

Index(['Categoria itinerario', 'Nome', 'Localitaˈ', 'Cap', 'Provincia',
       'Regione', 'Tipologia', 'Descrizione', 'Note', 'Url immagine',
       'Latitude', 'Longitude'],
      dtype='object')

In [6]:
itineraries_nocera_df.columns

Index(['Categoria itinerario', 'Città', 'Località', 'Via', 'Cap', 'Provincia',
       'Regione', 'N. Registro generale', 'Nome azienda', 'Data deposito',
       'Data registrazione marchio', 'Luogo di Deposito', 'Nome marchio',
       'Settore produzione', 'Importazione Si/No', 'Itinerario (periodo)',
       'Descrizione del marchio', 'Sito azienda', 'Note', 'Bibliografia',
       'Sitografia', 'URL immagine', 'URL immagine 2', 'Tipologia edificio',
       'Rifunzionalizzato Si/No', 'Tipo di rifunzionalizzazione', 'Latitude',
       'Longitude'],
      dtype='object')

In [7]:
itineraries_nocera_inferiore_df.columns

Index(['Categoria itinerario', 'Città', 'Località', 'Via', 'Cap', 'Provincia',
       'Regione', 'Sito/definizione manufatto', 'Epoca prima attestazione',
       'Itinerario (periodo)', 'Informazioni', 'URL immagine', 'Bibliografia',
       'Sitografia', 'Tipologia edificio', 'Rifunzionalizzato',
       'Tipo di rifunzionalizzazione', 'Latitude', 'Longitude'],
      dtype='object')

## Split and merge tables based on itinerary category

### Itineraries culturale

In [8]:
itineraries_culturale_caserta_df = itineraries_caserta_df[itineraries_caserta_df['Categoria itinerario'] == "culturale"]

column_mapping_culturale_caserta = {
    "luogo_interesse": None,
    "luogo_interesse_nome": 'Nome',
    "luogo_interesse_tipologia": 'Tipologia',
    "luogo_interesse_secolo_costruzione": 'Secolo di costruzione',
    "luogo_interesse_prima_attestazione": 'Anno prima attestazione',
    "luogo_interesse_stile": 'Stile',
    "comune": None,
    "comune_nome": 'Comune',
    "comune_cap": 'Cap',
    "comune_provincia": 'Provincia',
    "comune_regione": 'Regione',
    "via": 'Via',
    "localita": 'Località',
    "tipologia": 'Itinerario (periodo)',
    "informazioni": 'Informazioni',
    "note": None,
    "bibliografia": 'Bibliografia',
    "sitografia": 'Sitografia',
    "url_immagine": 'URL Immagine',
    "latitudine": 'Latitude',
    "longitudine": 'Longitude'
}

column_mapping_culturale_caserta_inv = {v: k for k, v in column_mapping_culturale_caserta.items() if v}

itineraries_culturale_caserta_df = itineraries_culturale_caserta_df.reindex(columns=column_mapping_culturale_caserta_inv.keys())\
                                                                    .rename(columns=column_mapping_culturale_caserta_inv)\
                                                                    .rename_axis(None,axis=1)

print(f"Columns that are not used:\n{set(itineraries_caserta_df.columns).difference(column_mapping_culturale_caserta.values())}")

Columns that are not used:
{'Tipo di Rifunzionalizzazione', 'Categoria itinerario', 'Rifunzionalizzato'}


In [9]:
itineraries_culturale_nocera_inferiore_df = itineraries_nocera_inferiore_df[itineraries_nocera_inferiore_df['Categoria itinerario'] == "culturale"]

column_mapping_culturale_nocera_inferiore = {
    "luogo_interesse": None,
    "luogo_interesse_nome": None, #could be "Sito/definizione manufatto"
    "luogo_interesse_tipologia": 'Tipologia edificio',
    "luogo_interesse_secolo_costruzione": None,
    "luogo_interesse_prima_attestazione": 'Epoca prima attestazione',
    "luogo_interesse_stile": None,
    "comune": None,
    "comune_nome": 'Città',
    "comune_cap": 'Cap',
    "comune_provincia": 'Provincia',
    "comune_regione": 'Regione',
    "via": 'Via',
    "localita": 'Località',
    "tipologia": 'Itinerario (periodo)',
    "informazioni": 'Informazioni',
    "note": None,
    "bibliografia": 'Bibliografia',
    "sitografia": 'Sitografia',
    "url_immagine": 'URL immagine',
    "latitudine": 'Latitude',
    "longitudine": 'Longitude'
}

column_mapping_culturale_nocera_inferiore_inv = {v: k for k, v in column_mapping_culturale_nocera_inferiore.items() if v}

itineraries_culturale_nocera_inferiore_df = itineraries_culturale_nocera_inferiore_df.reindex(columns=column_mapping_culturale_nocera_inferiore_inv.keys())\
                                                                    .rename(columns=column_mapping_culturale_nocera_inferiore_inv)\
                                                                    .rename_axis(None,axis=1)

print(f"Columns that are not used:\n{set(itineraries_nocera_inferiore_df.columns).difference(column_mapping_culturale_nocera_inferiore.values())}")

Columns that are not used:
{'Tipo di rifunzionalizzazione', 'Sito/definizione manufatto', 'Rifunzionalizzato', 'Categoria itinerario'}


In [10]:
itineraries_culturale_costiera_amalfitana_df = itineraries_costiera_amalfitana_df[itineraries_costiera_amalfitana_df['Categoria itinerario'] == "culturale"]

column_mapping_culturale_costiera_amalfitana = {
    "luogo_interesse": None,
    "luogo_interesse_nome": 'Nome',
    "luogo_interesse_tipologia": 'Tipologia',
    "luogo_interesse_secolo_costruzione": None,
    "luogo_interesse_prima_attestazione": None,
    "luogo_interesse_stile": None,
    "comune": None,
    "comune_nome": 'Localitaˈ',
    "comune_cap": 'Cap',
    "comune_provincia": 'Provincia',
    "comune_regione": 'Regione',
    "via": None,
    "localita": None,
    "tipologia": None,
    "informazioni": 'Descrizione',
    "note": 'Note',
    "bibliografia": None,
    "sitografia": None,
    "url_immagine": 'Url immagine',
    "latitudine": 'Latitude',
    "longitudine": 'Longitude'
}

column_mapping_culturale_costiera_amalfitana_inv = {v: k for k, v in column_mapping_culturale_costiera_amalfitana.items() if v}

itineraries_culturale_costiera_amalfitana_df = itineraries_culturale_costiera_amalfitana_df.reindex(columns=column_mapping_culturale_costiera_amalfitana_inv.keys())\
                                                                    .rename(columns=column_mapping_culturale_costiera_amalfitana_inv)\
                                                                    .rename_axis(None,axis=1)

print(f"Columns that are not used:\n{set(itineraries_costiera_amalfitana_df.columns).difference(column_mapping_culturale_costiera_amalfitana.values())}")

Columns that are not used:
{'Categoria itinerario'}


In [11]:
itineraries_culturale_df = pd.concat([itineraries_culturale_caserta_df, itineraries_culturale_nocera_inferiore_df, itineraries_culturale_costiera_amalfitana_df], ignore_index=True, sort=False)
itineraries_culturale_df = itineraries_culturale_df.fillna("NULL")


### Itineraries culinario

In [12]:
itineraries_culinario_costiera_amalfitana_df = itineraries_costiera_amalfitana_df[itineraries_costiera_amalfitana_df['Categoria itinerario'] == "culinario"]

column_mapping_culinario_costiera_amalfitana = {
    "nome": "Nome",
    "comune": None,
    "comune_nome": 'Localitaˈ',
    "comune_cap": 'Cap',
    "comune_provincia": 'Provincia',
    "comune_regione": 'Regione',
    "tipologia": "Tipologia",
    "descrizione": 'Descrizione',
    "note": 'Note',
    "url_immagine": 'Url immagine',
    "latitudine": 'Latitude',
    "longitudine": 'Longitude'
}

column_mapping_culinario_costiera_amalfitana_inv = {v: k for k, v in column_mapping_culinario_costiera_amalfitana.items() if v}

itineraries_culinario_costiera_amalfitana_df = itineraries_culinario_costiera_amalfitana_df.reindex(columns=column_mapping_culinario_costiera_amalfitana_inv.keys())\
                                                                    .rename(columns=column_mapping_culinario_costiera_amalfitana_inv)\
                                                                    .rename_axis(None,axis=1)

print(f"Columns that are not used:\n{set(itineraries_costiera_amalfitana_df.columns).difference(column_mapping_culinario_costiera_amalfitana.values())}")

Columns that are not used:
{'Categoria itinerario'}


In [13]:
itineraries_culinario_df = itineraries_culinario_costiera_amalfitana_df

### Itineraries aziendale

In [14]:
itineraries_aziendale_nocera_df = itineraries_nocera_df[itineraries_nocera_df['Categoria itinerario'] == "aziendale"]

column_mapping_aziendale_nocera = {
    "tipologia_itinerario": "Itinerario (periodo)",
    "comune": None,
    "comune_nome": 'Città',
    "comune_cap": 'Cap',
    "comune_provincia": 'Provincia',
    "comune_regione": 'Regione',    
    "localita": 'Località',
    "note": 'Note',
    "bibliografia": 'Bibliografia',
    "url_immagine": 'URL immagine',
    "sitografia": 'Sitografia',
    "azienda": None,
    "azienda_n_registro": 'N. Registro generale',
    "azienda_nome": 'Nome azienda',
    "azienda_data_deposito": 'Data deposito',
    "azienda_luogo_deposito": 'Luogo di Deposito',
    "azienda_settore_produzione": 'Settore produzione',
    "azienda_importazione": 'Importazione Si/No',
    "azienda_nome_marchio": 'Nome marchio',
    "azienda_data_registrazione_marchio": 'Data registrazione marchio',
    "azienda_descrizione_marchio": 'Descrizione del marchio',
    "latitudine": 'Latitude',
    "longitudine": 'Longitude'
}

column_mapping_aziendale_nocera_inv = {v: k for k, v in column_mapping_aziendale_nocera.items() if v}

itineraries_aziendale_nocera_df = itineraries_aziendale_nocera_df.reindex(columns=column_mapping_aziendale_nocera_inv.keys())\
                                                                    .rename(columns=column_mapping_aziendale_nocera_inv)\
                                                                    .rename_axis(None,axis=1)

print(f"Columns that are not used:\n{set(itineraries_nocera_df.columns).difference(column_mapping_aziendale_nocera.values())}")

Columns that are not used:
{'Tipo di rifunzionalizzazione', 'Via', 'URL immagine 2', 'Tipologia edificio', 'Categoria itinerario', 'Rifunzionalizzato Si/No', 'Sito azienda'}


In [15]:
itineraries_aziendale_df = itineraries_aziendale_nocera_df

## Extract join tables and foreign keys

In [16]:
def revert_join(df_list, prefix_columns_to_exclude=None, columns_to_exclude=None, foreign_key_name="fk_id", delete_prefix=True):
    if columns_to_exclude is None:
        columns_to_exclude = [[] for _ in range(len(df_list))]
    
    if prefix_columns_to_exclude:
        for i, df in enumerate(df_list):
            columns_to_exclude[i].extend([col for col in df if col.startswith(prefix_columns_to_exclude)])
        if foreign_key_name == "fk_id":
            foreign_key_name = f"{prefix_columns_to_exclude}_id"
    
    join_table_df_list = []
    
    for i, df in enumerate(df_list):
        temp = pd.DataFrame(df.groupby(columns_to_exclude[i])[columns_to_exclude[i][0]].count())
        temp.columns = ["dummy"]
        join_table_df_list.append(temp)
    join_table_df = pd.concat(join_table_df_list)
    join_table_df.columns=['dummy']
    join_table_df = join_table_df.reset_index().drop('dummy', axis=1)
    join_table_df = join_table_df.drop_duplicates()
    join_table_df.index.name=foreign_key_name
    for i, df in enumerate(df_list):
        new_columns = [column for column in df.columns if column not in columns_to_exclude[i]]
        new_columns.append(foreign_key_name)
        df_list[i] = df.merge(join_table_df.reset_index())[new_columns]
    join_table_df.index.name='id'
    if prefix_columns_to_exclude is not None and delete_prefix:
        join_table_df.columns = join_table_df.columns.str.replace(prefix_columns_to_exclude, "")
        join_table_df.columns = join_table_df.columns.str.lstrip("_")
    return df_list, join_table_df

### Table comune

In [17]:
df_list, comune_df = revert_join([itineraries_culturale_df, itineraries_culinario_df, itineraries_aziendale_df],\
                        prefix_columns_to_exclude="comune")
(itineraries_culturale_no_comune_df, itineraries_culinario_no_comune_df, itineraries_aziendale_no_comune_df) = df_list

### Table azienda

In [18]:
(itineraries_aziendale_no_comune_no_azienda_df, ), azienda_df = revert_join([itineraries_aziendale_no_comune_df],\
                        prefix_columns_to_exclude="azienda")

### Table luogo_di_interesse

In [19]:
(itineraries_culturale_no_comune_no_luogo_di_interesse_df, ), luogo_di_interesse_df = revert_join([itineraries_culturale_no_comune_df],\
                        prefix_columns_to_exclude="luogo_interesse")

## Add index as id-column

In [20]:
def add_column_id_from_index(df):
    df.index.name = 'id'
    return df.reset_index()

comune_df = add_column_id_from_index(comune_df)
azienda_df = add_column_id_from_index(azienda_df)
luogo_di_interesse_df = add_column_id_from_index(luogo_di_interesse_df)
itineraries_aziendale_no_comune_no_azienda_df = add_column_id_from_index(itineraries_aziendale_no_comune_no_azienda_df)
itineraries_culinario_no_comune_df = add_column_id_from_index(itineraries_culinario_no_comune_df)
itineraries_culturale_no_comune_no_luogo_di_interesse_df = add_column_id_from_index(itineraries_culturale_no_comune_no_luogo_di_interesse_df)

## Check columns

In [21]:
azienda_df.columns

Index(['id', 'n_registro', 'nome', 'data_deposito', 'luogo_deposito',
       'settore_produzione', 'importazione', 'nome_marchio',
       'data_registrazione_marchio', 'descrizione_marchio'],
      dtype='object')

In [22]:
comune_df.columns

Index(['id', 'nome', 'cap', 'provincia', 'regione'], dtype='object')

In [23]:
luogo_di_interesse_df.columns

Index(['id', 'nome', 'tipologia', 'secolo_costruzione', 'prima_attestazione',
       'stile'],
      dtype='object')

In [24]:
itineraries_aziendale_no_comune_no_azienda_df.columns

Index(['id', 'tipologia_itinerario', 'localita', 'note', 'bibliografia',
       'url_immagine', 'sitografia', 'latitudine', 'longitudine', 'comune_id',
       'azienda_id'],
      dtype='object')

In [25]:
itineraries_culinario_no_comune_df.columns

Index(['id', 'nome', 'tipologia', 'descrizione', 'note', 'url_immagine',
       'latitudine', 'longitudine', 'comune_id'],
      dtype='object')

In [26]:
itineraries_culturale_no_comune_no_luogo_di_interesse_df.columns

Index(['id', 'via', 'localita', 'tipologia', 'informazioni', 'bibliografia',
       'sitografia', 'url_immagine', 'latitudine', 'longitudine', 'note',
       'comune_id', 'luogo_interesse_id'],
      dtype='object')

## Populate database with tables

In [27]:
azienda_df = azienda_df.replace('NULL', None)
comune_df = comune_df.replace('NULL', None)
luogo_di_interesse_df = luogo_di_interesse_df = luogo_di_interesse_df.replace('NULL', None)
itineraries_aziendale_no_comune_no_azienda_df = itineraries_aziendale_no_comune_no_azienda_df.replace('NULL', None)
itineraries_culinario_no_comune_df = itineraries_culinario_no_comune_df.replace('NULL', None)
itineraries_culturale_no_comune_no_luogo_di_interesse_df = itineraries_culturale_no_comune_no_luogo_di_interesse_df.replace('NULL', None)

engine = create_engine(f'{dialect}://{user}:{password}@{host}:{port}/{database}', echo=False)
conn = engine.connect()
#Session = sessionmaker(bind=engine) 
#with Session() as session:

azienda_df.to_sql(name='azienda', schema='public', con=engine, if_exists='append', index=False)
comune_df.to_sql(name='comune', schema='public', con=engine, if_exists='append', index=False)
luogo_di_interesse_df.to_sql(name='luogo_di_interesse', schema='public', con=engine, if_exists='append', index=False)
itineraries_aziendale_no_comune_no_azienda_df.to_sql(name='itinerario_aziendale', schema='public', con=engine, if_exists='append', index=False)
itineraries_culinario_no_comune_df.to_sql(name='itinerario_culinario', schema='public', con=engine, if_exists='append', index=False)
itineraries_culturale_no_comune_no_luogo_di_interesse_df.to_sql(name='itinerario_culturale', schema='public', con=engine, if_exists='append', index=False)

conn.close()